In [1]:
import os,sys
sys.path.append("%s/analysis-CommonTools/"%os.getenv('HOME'))
from common_imports import *
from matplotlib_tools import *

In [2]:
import ROOT

Welcome to JupyROOT 6.22/08


In [3]:
ROOT.EnableImplicitMT()

In [4]:
%jsroot on

In [5]:
def prepare_df(df,selection):
    
    df_sel = df.Filter(selection)
    
    df_sel = df_sel.Define("angle_azimuth","atan(dir.x/dir.z)")
    df_sel = df_sel.Define("angle_zenith","atan(sqrt(dir.x*dir.x+dir.y*dir.y)/dir.z)")

    df_planes = {}
    
    for p in range(3):
        #hitstr="hits%d"%plane
        df_sel = df_sel.Define(f"hits{p}_n",
                               f"hits{p}.dir.x.size()")
        df_planes[p] = df_sel.Filter(f"hits{p}_n > 0")
        df_planes[p] = df_planes[p].Define(f"hits{p}_angle_azimuth",
                                           f"atan(hits{p}.dir.x.back()/hits{p}.dir.z.back())")
        df_planes[p] = df_planes[p].Define(f"hits{p}_angle_zenith",
                                           f"atan(sqrt(hits{p}.dir.x.back()*hits{p}.dir.x.back()+hits{p}.dir.y.back()*hits{p}.dir.y.back())/hits{p}.dir.z.back())")
        for var in ["dqdx"]:
            df_planes[p] = df_planes[p].Define(f"hits{p}_{var}_mean",
                                               f"ROOT::VecOps::Mean(hits{p}.{var})")
        for hvar in ["integral","width"]:
            df_planes[p] = df_planes[p].Define(f"hits{p}_{hvar}_mean",
                                               f"ROOT::VecOps::Mean(hits{p}.h.{hvar})")
    
    return df_sel, df_planes

In [6]:
def create_histo(df,var,hist_registry):
    if var in hist_registry:
        return df.Histo1D(hist_registry[var],var)
    else:
        return df.Histo1D(var)

In [7]:
def make_histos_dict(df,selection,hr):
    
    df_sel, df_sel_planes = prepare_df(df,selection)
    
    hists = {}
    
    for c in ["x","y","z"]:
        hists[f"start_{c}"] = create_histo(df_sel,f"start.{c}",hr)
        hists[f"end_{c}"] = create_histo(df_sel,f"end.{c}",hr)
        hists[f"dir_{c}"] = create_histo(df_sel,f"dir.{c}",hr)
    
    hists["length"] = create_histo(df_sel,"length",hr)
    hists["t0"] = create_histo(df_sel,"t0",hr)
    hists["angle_azimuth"] = create_histo(df_sel,"angle_azimuth",hr)
    hists["angle_zenith"] = create_histo(df_sel,"angle_zenith",hr)
    
    for p in range(3):
        for v in ["dqdx_mean","integral_mean","width_mean","angle_azimuth","angle_zenith","n"]:
            hists[f"hits{p}_{v}"] = create_histo(df_sel_planes[p],f"hits{p}_{v}",hr)
    
    return hists, df_sel, df_sel_planes

In [39]:
def compare_histos(hist_names,histos_1,histos_2):
    for name in hist_names:
        if name in histos_1.keys() and name in histos_2.keys(): 
            print(f"{name}: Chi2Test={histos_1[name].Chi2Test(histos_2[name].GetPtr(),'NORM')}")
        else:
            print(f"Couldn't find {name} in both histogram sets.")

In [41]:
def draw_histos(hname,hists_data,hists_mc):
    hists_mc[hname].SetLineColor(ROOT.kBlue)
    hists_data[hname].SetLineColor(ROOT.kRed)

    hists_data[hname].DrawNormalized()
    hists_mc[hname].DrawNormalized("SAME")
    
    return c.Draw()

In [8]:
hist_registry = {}

hist_registry["start_x"] = ("","Start X;x (cm);Tracks",100,-400,400)
hist_registry["start_y"] = ("","Start Y;y (cm);Tracks",100,-150,150)
hist_registry["start_z"] = ("","Start Z;z (cm);Tracks",100,-1500,1500)

hist_registry["end_x"] = ("","End X;x (cm);Tracks",100,-400,400)
hist_registry["end_y"] = ("","End Y;y (cm);Tracks",100,-150,150)
hist_registry["end_z"] = ("","End Z;z (cm);Tracks",100,-1500,1500)

hist_registry["length"] = ("","Length;length (cm);Tracks",100,0,1500)
hist_registry["t0"] = ("","T_{0};time (#mus);Tracks",100,-1.5e6,1.5e6)
hist_registry["angle_azimuth"] = ("","Azimuthal Angle;azimuth (rad);Tracks",100,-0.5*np.pi,0.5*np.pi)
hist_registry["angle_zenith"] = ("","Zenith Angle;zenith (rad);Tracks",100,-0.5*np.pi,0.5*np.pi)

hist_registry["hits0_dqdx_mean"] = ("","Mean dQdx (Plane 0);dqdx;Tracks",100,0,1500)
hist_registry["hits1_dqdx_mean"] = ("","Mean dQdx (Plane 1);dqdx;Tracks",100,0,1500)
hist_registry["hits2_dqdx_mean"] = ("","Mean dQdx (Plane 2);dqdx;Tracks",100,0,1500)

hist_registry["hits0_integral_mean"] = ("","Mean Integral Charge (Plane 0);charge (adc);Tracks",100,0,1000)
hist_registry["hits1_integral_mean"] = ("","Mean Integral Charge (Plane 1);charge (adc);Tracks",100,0,1000)
hist_registry["hits2_integral_mean"] = ("","Mean Integral Charge (Plane 2);charge (adc);Tracks",100,0,1000)

hist_registry["hits0_width_mean"] = ("","Mean Hit Width (Plane 0);width (ticks);Tracks",100,0,10)
hist_registry["hits1_width_mean"] = ("","Mean Hit Width (Plane 1);width (ticks);Tracks",100,0,10)
hist_registry["hits2_width_mean"] = ("","Mean Hit Width (Plane 2);width (ticks);Tracks",100,0,10)


In [9]:
#bnb_sim_files_dir = "/Users/wketchum/Data/ICARUS/CalibrationNtuples_10Mar2022/ICARUS_BNB_Nu_Cosmics"
#bnb_data_files_dir = "/Users/wketchum/Data/ICARUS/CalibrationNtuples_10Mar2022/ICARUS_BNB_Data"

bnb_sim_files_dir = "/pnfs/icarus/persistent/calibration/calib_ntuples/mc/ICARUS_BNB_Nu_Cosmics"
bnb_data_files_dir = "/pnfs/icarus/persistent/calibration/calib_ntuples/data"

In [10]:
c = ROOT.TCanvas()

In [11]:
selection_stopping = "selected==0"

In [12]:
files_bnb_sim = ROOT.TFileCollection()
files_bnb_sim.Add(f"{bnb_sim_files_dir}/hist_prodcorsika_bnb*.root")
print("Found %d BNB sim files in our list."%files_bnb_sim.GetNFiles())

Found 2291 BNB sim files in our list.


In [13]:
files_bnb_data = ROOT.TFileCollection()
files_bnb_data.Add(f"{bnb_data_files_dir}/hist_data_dl4_fstrmBNB*.root")
print("Found %d BNB data files in our list."%files_bnb_data.GetNFiles())

Found 787 BNB data files in our list.


In [14]:
chain_bnb_sim = ROOT.TChain("caloskimE/TrackCaloSkim");
chain_bnb_sim.AddFileInfoList(files_bnb_sim.GetList())

1

In [15]:
chain_bnb_data = ROOT.TChain("caloskimE/TrackCaloSkim");
chain_bnb_data.AddFileInfoList(files_bnb_data.GetList())

1

In [16]:
#df_bnb_sim_stopping = prepare_df(ROOT.RDataFrame(chain_bnb_sim),selection_stopping)
hists_bnb_sim_stopping, df_sim_stopping, df_sim_stopping_planes = make_histos_dict(ROOT.RDataFrame(chain_bnb_sim),
                                                                                   selection_stopping,
                                                                                   hist_registry)

In [17]:
#df_bnb_data_stopping = prepare_df(ROOT.RDataFrame(chain_bnb_data),selection_stopping)
hists_bnb_data_stopping, df_data_stopping, df_data_stopping_planes = make_histos_dict(ROOT.RDataFrame(chain_bnb_data),
                                                                                      selection_stopping,
                                                                                      hist_registry)

In [18]:
#ROOT.RDF.SaveGraph(hists_bnb_sim_stopping["hits2_dqdx_mean"])

In [19]:
h_mc = hists_bnb_sim_stopping["hits2_dqdx_mean"]
h_data = hists_bnb_data_stopping["hits2_dqdx_mean"]

h_mc.SetLineColor(ROOT.kBlue)
h_data.SetLineColor(ROOT.kRed)

h_data.DrawNormalized()
h_mc.DrawNormalized("SAME")
c.Draw()

In [31]:
h_mc = hists_bnb_sim_stopping["length"]
h_data = hists_bnb_data_stopping["length"]

h_mc.SetLineColor(ROOT.kBlue)
h_data.SetLineColor(ROOT.kRed)

h_data.DrawNormalized()
h_mc.DrawNormalized("SAME")
c.Draw()

In [40]:
compare_histos(hist_registry.keys(),hists_bnb_data_stopping,hists_bnb_sim_stopping)

start_x: Chi2Test=5.2263975736268366e-138
start_y: Chi2Test=0.0
start_z: Chi2Test=0.24867733882135964
end_x: Chi2Test=2.552081612407592e-14
end_y: Chi2Test=0.00021383343323406377
end_z: Chi2Test=0.41294793178485373
length: Chi2Test=0.07310588749362035
t0: Chi2Test=5.02087026260076e-19
angle_azimuth: Chi2Test=8.732907851667684e-51
angle_zenith: Chi2Test=0.4950297403972618
hits0_dqdx_mean: Chi2Test=2.5585030778894925e-76
hits1_dqdx_mean: Chi2Test=0.0
hits2_dqdx_mean: Chi2Test=3.43981720282712e-67
hits0_integral_mean: Chi2Test=5.114466277524704e-90
hits1_integral_mean: Chi2Test=1.4221167120111905e-278
hits2_integral_mean: Chi2Test=1.773855630767473e-05
hits0_width_mean: Chi2Test=4.813538217319947e-184
hits1_width_mean: Chi2Test=1.2817877102823032e-08
hits2_width_mean: Chi2Test=1.8354912034355545e-62


In [33]:
h_data.Chi2Test(h_mc.GetPtr(),"NORM")

0.07310588749362035

In [48]:
draw_histos("hits2_width_mean",hists_bnb_data_stopping,hists_bnb_sim_stopping)

In [49]:
df_data_stopping_planes[2].Profile1D("angle_azimuth","hits2_width_mean").Draw()

TypeError: Template method resolution failed:
  none of the 2 overloaded methods succeeded. Full details:
  ROOT::RDF::RResultPtr<TProfile> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Profile1D(const ROOT::RDF::TProfile1DModel& model, basic_string_view<char,char_traits<char> > v1Name, basic_string_view<char,char_traits<char> > v2Name, basic_string_view<char,char_traits<char> > wName) =>
    TypeError: takes at least 4 arguments (2 given)
  ROOT::RDF::RResultPtr<TProfile> ROOT::RDF::RInterface<ROOT::Detail::RDF::RJittedFilter,void>::Profile1D(const ROOT::RDF::TProfile1DModel& model, basic_string_view<char,char_traits<char> > v1Name = "", basic_string_view<char,char_traits<char> > v2Name = "") =>
    TypeError: could not convert argument 1
  Failed to instantiate "Profile1D(std::string,std::string)"

In [51]:
df_data_stopping_planes[2].Histo1D("angle_azimuth")

<cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> object at 0x20b9fa50>